In [28]:
import subprocess, json

BASE = "http://127.0.0.1:18080"

def call_api(path: str):
    result = subprocess.run(["curl", "-s", f"{BASE}{path}"], capture_output=True, text=True)
    raw = result.stdout.strip()
    try:
        print(json.dumps(json.loads(raw), indent=2))
    except json.JSONDecodeError:
        print(raw)


In [29]:
call_api("/health")

{
  "status": "ok"
}


In [30]:
call_api("/db-check")

{
  "ok": true,
  "db_time": "2025-08-30T21:37:38.878284+02:00"
}


In [34]:
call_api("/servers")

[
  {
    "id": 1,
    "hostname": "myserver.local.lan",
    "configuration": {},
    "datacenter_id": 1,
    "created_at": "2024-05-28T11:39:01.516030",
    "modified_at": "2024-05-28T11:39:01.516030"
  },
  {
    "id": 2,
    "hostname": "database.local.lan",
    "configuration": {
      "user_limit": 10,
      "max_connections": 500
    },
    "datacenter_id": 2,
    "created_at": "2024-05-28T11:40:13.349528",
    "modified_at": "2024-05-28T11:40:13.349528"
  },
  {
    "id": 3,
    "hostname": "rabbitmq.local.lan",
    "configuration": {
      "max_queues": 1234
    },
    "datacenter_id": 2,
    "created_at": "2024-05-28T11:41:14.626643",
    "modified_at": "2024-05-28T11:41:14.626643"
  }
]


In [38]:
call_api("/servers?datacenter_id=2&sort=hostname")

[
  {
    "id": 2,
    "hostname": "database.local.lan",
    "configuration": {
      "user_limit": 10,
      "max_connections": 500
    },
    "datacenter_id": 2,
    "created_at": "2024-05-28T11:40:13.349528",
    "modified_at": "2024-05-28T11:40:13.349528"
  },
  {
    "id": 3,
    "hostname": "rabbitmq.local.lan",
    "configuration": {
      "max_queues": 1234
    },
    "datacenter_id": 2,
    "created_at": "2024-05-28T11:41:14.626643",
    "modified_at": "2024-05-28T11:41:14.626643"
  }
]


In [33]:
call_api("/servers/2")

{
  "id": 2,
  "hostname": "database.local.lan",
  "configuration": {
    "user_limit": 10,
    "max_connections": 500
  },
  "datacenter_id": 2,
  "created_at": "2024-05-28T11:40:13.349528",
  "modified_at": "2024-05-28T11:40:13.349528"
}
